In [1]:
%load_ext autoreload
%autoreload 2

import sys
if "/home/alee/psi4_mod/build/stage/lib" not in sys.path:
    sys.path.append("/home/alee/psi4_mod/build/stage/lib")
    print(sys.path)
import psi4

import math
import numpy as np

import torch
import torch_geometric

from utils import get_iso_permuted_dataset
from utils import flatten_list

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print (device)

torch.set_default_dtype(torch.float32)

['/home/alee/bdna/ml-dna-gitupload/potential', '/home/alee/anaconda3/envs/e3nn_psi4/lib/python37.zip', '/home/alee/anaconda3/envs/e3nn_psi4/lib/python3.7', '/home/alee/anaconda3/envs/e3nn_psi4/lib/python3.7/lib-dynload', '', '/home/alee/anaconda3/envs/e3nn_psi4/lib/python3.7/site-packages', '/home/alee/anaconda3/envs/e3nn_psi4/lib/python3.7/site-packages/IPython/extensions', '/home/alee/.ipython', '/home/alee/psi4_mod/build/stage/lib']
cuda


In [5]:
# first, get dataset

hhh = "h_s_only_augccpvdz_density.out"
ooo = "o_s_only_augccpvdz_density.out"
ccc = "c_s_only_augccpvdz_density.out"
nnn = "n_s_only_augccpvdz_density.out"
ppp = "p_s_only_augccpvdz_density.out"

test_datafile = "2mer-test.pkl"
test_dataset = get_iso_permuted_dataset(test_datafile,h_iso=hhh,c_iso=ccc,n_iso=nnn,o_iso=ooo,p_iso=ppp)

b = 1

test_loader = torch_geometric.data.DataLoader(test_dataset, batch_size=b, shuffle=False)


In [3]:
from e3nn.nn.models.gate_points_2101 import Network
from e3nn import o3

model_kwargs = {
    "irreps_in": "5x 0e", #irreps_in 
    "irreps_hidden": [(mul, l, p) for l, mul in enumerate([200,67,40,29]) for p in [-1, 1]], #irreps_hidden
    #"irreps_hidden": "100x0e + 100x0o",
    "irreps_out": "14x0e + 5x1o + 5x2e + 2x3o + 1x4e", #irreps_out
    "irreps_node_attr": None, #irreps_node_attr
    "irreps_edge_attr": o3.Irreps.spherical_harmonics(3), #irreps_edge_attr
    "layers": 5,
    "max_radius": 3.5,
    "num_neighbors": 12.666666,
    "number_of_basis": 10,
    "radial_layers": 1,
    "radial_neurons": 128,
    "num_nodes": 24,
    "reduce_output": False,
}

model = Network(**model_kwargs)

optim = torch.optim.Adam(model.parameters(), lr=1e-2)
optim.zero_grad()

model.to(device)

Network(
  (layers): ModuleList(
    (0): Compose(
      (first): Convolution(
        (sc): FullyConnectedTensorProduct(5x0e x 1x0e -> 336x0e+67x1o+40x2e+29x3o | 1680 paths | 1680 weights)
        (lin1): FullyConnectedTensorProduct(5x0e x 1x0e -> 5x0e | 25 paths | 25 weights)
        (fc): FullyConnectedNet[10, 128, 20]
        (tp): TensorProduct(5x0e x 1x0e+1x1o+1x2e+1x3o -> 5x0e+5x1o+5x2e+5x3o | 20 paths | 20 weights)
        (lin2): FullyConnectedTensorProduct(5x0e+5x1o+5x2e+5x3o x 1x0e -> 336x0e+67x1o+40x2e+29x3o | 2360 paths | 2360 weights)
      )
      (second): Gate (336x0e+67x1o+40x2e+29x3o -> 200x0e+67x1o+40x2e+29x3o)
    )
    (1): Compose(
      (first): Convolution(
        (sc): FullyConnectedTensorProduct(200x0e+67x1o+40x2e+29x3o x 1x0e -> 472x0e+67x1o+67x1e+40x2o+40x2e+29x3o+29x3e | 101330 paths | 101330 weights)
        (lin1): FullyConnectedTensorProduct(200x0e+67x1o+40x2e+29x3o x 1x0e -> 200x0e+67x1o+40x2e+29x3o | 46930 paths | 46930 weights)
        (fc): FullyCo

In [4]:
checkpoint = torch.load('trainall-400.pkl')

model.load_state_dict(checkpoint)

<All keys matched successfully>

In [6]:
# generate density on a grid for a random structure
from utils import generate_grid, gau2grid_density_kdtree

num = 0

data = test_loader.dataset[num]

# run model on that sample
mask = torch.where(data.y == 0, torch.zeros_like(data.y), torch.ones_like(data.y)).detach()
y_ml = model(data.to(device))*mask.to(device)

x,y,z,vol,x_spacing,y_spacing,z_spacing = generate_grid(data,spacing=0.2,buffer=5.0)

Rs = [(14, 0), (5, 1), (5, 2), (2, 3), (1, 4)]

target_density, ml_density = gau2grid_density_kdtree(x.flatten(),y.flatten(),z.flatten(),data,y_ml,Rs)
print("done!")

small:  1e-05
done!


In [7]:
angstrom2bohr = 1.8897259886
print("old vol: ", vol)
vol = x_spacing * y_spacing * z_spacing
print("Read in: vol, x, y, z spacing", vol, x_spacing, y_spacing, z_spacing)

num_ele_ml = np.sum(ml_density)*vol*angstrom2bohr**3
num_ele_target = np.sum(target_density)*vol*angstrom2bohr**3

print("Predicted:", num_ele_ml)
print("Target:", num_ele_target)
print()

ep_ml = 100 * np.sum(np.abs(ml_density.flatten()-target_density.flatten())) / np.sum(target_density.flatten())

print("ML - target eps: ",ep_ml)


old vol:  0.007901703819858148
Read in: vol, x, y, z spacing 0.007901703819858148 0.22112696511404906 0.16338748023623495 0.21870578281463082
Predicted: 586.5680240768182
Target: 586.3934379497184

ML - target eps:  0.6439457943062418


In [8]:
## Now compute isosurface with marching cubes

from skimage import measure

l = round((ml_density.shape[0])**(1/3))
t_density = target_density.reshape(l,l,l)
m_density = ml_density.reshape(l,l,l)

# level sets what the density isosurface is
level = 0.01
t_verts, t_faces, t_normals, t_values = measure.marching_cubes(t_density, spacing=(x_spacing, y_spacing, z_spacing), level=level, step_size=2)
ml_verts, ml_faces, ml_normals, ml_values = measure.marching_cubes(m_density, spacing=(x_spacing, y_spacing, z_spacing), level=level, step_size=2)

# now scale to get back to real coordinates
xyz_min = np.array([x[0,0,0],y[0,0,0],z[0,0,0]])
t_verts = t_verts + xyz_min
ml_verts = ml_verts + xyz_min
print(ml_verts[:,0].shape)

print(ml_verts)


(8079,)
[[19.45853995 29.85462425 35.72275443]
 [19.49453981 29.85462425 35.65439386]
 [19.49453981 29.82478564 35.72275443]
 ...
 [38.660705   29.85462425 23.4752306 ]
 [38.58468079 29.85462425 23.91264216]
 [38.55121618 30.18139921 23.03781903]]


In [10]:
# compute the electrostatic potentials

from utils import compute_amber_potential
from utils import compute_potential_field_ml
from utils import compute_potential_field_target
m_isopot, m_isofield = compute_potential_field_ml(ml_verts[:,0].flatten(),ml_verts[:,1].flatten(),ml_verts[:,2].flatten(),data,y_ml,Rs, isoOnlyFlag=0)

In [11]:
# plotting: get atoms

h_x = []
h_y = []
h_z = []
c_x = []
c_y = []
c_z = []
n_x = []
n_y = []
n_z = []
o_x = []
o_y = []
o_z = []
p_x = []
p_y = []
p_z = []

for i in range(len(data.z)):
    if(data.z[i])==1:
        h_x.append(data.pos_orig[i][0].cpu().numpy())
        h_y.append(data.pos_orig[i][1].cpu().numpy())
        h_z.append(data.pos_orig[i][2].cpu().numpy())
    elif(data.z[i])==6:
        c_x.append(data.pos_orig[i][0].cpu().numpy())
        c_y.append(data.pos_orig[i][1].cpu().numpy())
        c_z.append(data.pos_orig[i][2].cpu().numpy())
    elif(data.z[i])==7:
        n_x.append(data.pos_orig[i][0].cpu().numpy())
        n_y.append(data.pos_orig[i][1].cpu().numpy())
        n_z.append(data.pos_orig[i][2].cpu().numpy())
    elif(data.z[i])==8:
        o_x.append(data.pos_orig[i][0].cpu().numpy())
        o_y.append(data.pos_orig[i][1].cpu().numpy())
        o_z.append(data.pos_orig[i][2].cpu().numpy())
    elif(data.z[i])==15:
        p_x.append(data.pos_orig[i][0].cpu().numpy())
        p_y.append(data.pos_orig[i][1].cpu().numpy())
        p_z.append(data.pos_orig[i][2].cpu().numpy())
    else:
        print("Atom type not supported!")

In [12]:
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# FigureWidget apparently works faster with numpy arrays
fig = go.FigureWidget()

traces = []
for verts, faces, intense, name in zip([ml_verts], [ml_faces], [m_isopot], ['Density']):
    traces.append(go.Mesh3d(
        x=verts[:,0].flatten(),
        y=verts[:,1].flatten(),
        z=verts[:,2].flatten(),
        i=faces[:,0].flatten(),
        j=faces[:,1].flatten(),
        k=faces[:,2].flatten(),
        intensity=intense.flatten(),
        colorscale="RdBu",
        colorbar=dict(tickfont=dict(size=20, color='black', family='Arial')),
        opacity=0.8,
        name=name,
    ))

fig.add_trace(traces[0])

h_geom = go.Scatter3d(x=h_x,y=h_y,z=h_z,mode='markers',marker=dict(size=4.6,color='gray',opacity=1.0), showlegend=False)
fig.add_trace(h_geom)
c_geom = go.Scatter3d(x=c_x,y=c_y,z=c_z,mode='markers',marker=dict(size=7.7,color='black',opacity=1.0), showlegend=False)
fig.add_trace(c_geom)
n_geom = go.Scatter3d(x=n_x,y=n_y,z=n_z,mode='markers',marker=dict(size=7.4,color='blue',opacity=1.0), showlegend=False)
fig.add_trace(n_geom)
o_geom = go.Scatter3d(x=o_x,y=o_y,z=o_z,mode='markers',marker=dict(size=7.4,color='red',opacity=1.0), showlegend=False)
fig.add_trace(o_geom)
p_geom = go.Scatter3d(x=p_x,y=p_y,z=p_z,mode='markers',marker=dict(size=11,color='orange',opacity=1.0), showlegend=False)
fig.add_trace(p_geom)

fig.write_html("./2mertest0-pot.html")

#fig.show()